In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
sys.path.append("..")

In [2]:
from abautomator import get_query, experiment

In [3]:
exp = experiment.Experiment(
        ctrl_cond='Dec1021InspirationMomentFinalControl',
        tx_conds=[
            'Dec1021InspirationMomentFinalVideo01',
            'Dec1021InspirationMomentFinalVideo02',
            'Dec1021InspirationMomentFinalCarousel01',
            'Dec1021InspirationMomentFinalCarousel02',
            'Dec1021InspirationMomentFinalCarousel03',
            'Dec1021InspirationMomentFinalCarousel04',
        ],
        event="segment_signup_flow_started",
        event_prop="context_traits_onboarding_flow_001",
        start_dt=date.today() - timedelta(days=2),
    )

In [4]:
name = "sampling_dist"
df = pickle.load(
            open(os.path.join("..", "tests", f"{name}.p"), "rb" )
        )

In [5]:
df.head(2)

,exp_cond,metric,metric_cond_label,est_mean,est_se
0,Video01,n_user_sessions,"(n_user_sessions, Video01)",0.014278,0.066013
1,Video01,pct_user_sessions,"(pct_user_sessions, Video01)",0.009449,0.007249


In [138]:
def add_confidence_intervals(df):
    df["upper_68_ci"] = df["est_mean"] + df["est_se"]
    df["lower_68_ci"] = df["est_mean"] - df["est_se"]

    df["upper_95_ci"] = df["est_mean"] + (2 * df["est_se"])
    df["lower_95_ci"] = df["est_mean"] - (2 * df["est_se"])
    return df

In [76]:
import math

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Colorblind6
from bokeh.transform import factor_cmap

from math import pi

output_notebook()

Loading BokehJS ...

In [139]:
df = add_confidence_intervals(df)

In [140]:
source = ColumnDataSource(df)

p = figure(
    y_range=FactorRange(*source.data["metric_cond_label"]),
    height=450,
    title=f"{exp.name}",
    toolbar_location=None,
    tools=""
)

p.segment(
    source.data["upper_95_ci"],
    source.data["metric_cond_label"],
    source.data["upper_68_ci"],
    source.data["metric_cond_label"],
    color="black",
)

p.segment(
    source.data["lower_95_ci"],
    source.data["metric_cond_label"],
    source.data["lower_68_ci"],
    source.data["metric_cond_label"],
    color="black",
)

p.hbar(
    y='metric_cond_label',
    right="upper_68_ci",
    left="lower_68_ci",
    legend_field='exp_cond',
    fill_color=factor_cmap(
        'exp_cond', palette=Colorblind6, factors=list(set(source.data["exp_cond"])), start=1, end=2
    ),
    line_color=None,
    height=0.6,
    fill_alpha=0.8,
    source=source,
)

p.legend.location = "top_right"

p.ygrid.grid_line_color = None
p.xaxis.axis_label = 'Absolute difference from control'

p.yaxis.major_label_text_alpha = 0.0
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 0
p.yaxis.major_label_text_font_size = '1px'

p.yaxis.group_label_orientation = "horizontal"
p.yaxis.separator_line_alpha = 0
p.yaxis.minor_tick_line_color = "yellow"

In [141]:
show(p)